In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Import libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split

## Importing data

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/test.csv')

In [ ]:
train.info()
print('-'*70)
print('-'*70)
test.info()

In [ ]:
train.describe()

In [ ]:
test.describe()

## Looking at the distribution of each feature

In [ ]:
fig, axes = plt.subplots(5,3,figsize=(16,15))
plt.subplots_adjust(hspace=0.4, wspace=0.3)

for i in range(14):
    sns.histplot(train['cont' + str(i+1)], ax=axes[i//3, i%3])
    axes[i//3, i%3].set(title='cont' + str(i+1), xlabel='')
    
sns.histplot(train['target'], ax=axes[4,2])
axes[4,2].set(title='target', xlabel='')

plt.show()

The feature 'cont5' seems to be very distorted as more than $1/3^{rd}$ of the values are concentrated around zero. So, we can ignore that column, neglecting its effect on the prediction of the target value.

In [ ]:
train.drop(['id', 'cont5'], axis=1, inplace=True)
test.drop('cont5', axis=1, inplace=True)

Here, we will split the training dataset into training \& validation sets.

In [ ]:
train_x, val_x, train_y, val_y = train_test_split(train.drop(['target'], axis=1), 
                                                  train.target, 
                                                  test_size=0.2)

## NN Model

In [ ]:
model = tf.keras.models.Sequential([tf.keras.layers.Dense(64, input_shape=(13,)),
                                    tf.keras.layers.Dense(32, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(16, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(1, activation=tf.nn.relu)])

model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.MeanSquaredError())

model.fit(train_x, train_y, validation_data=(val_x, val_y), epochs=10)

In [ ]:
model.summary()

In [ ]:
pred = model.predict(test.drop('id', axis=1))
pred.shape

In [ ]:
sub = pd.DataFrame({'id':list(test['id']) , 'target':pred[:,0]})
sub

In [ ]:
sub.to_csv('./submission.csv', index=False)